# Dataset Processing:
## Takes two datasets (OpenBCI data & and Keylogger), cleans, merges based on timestamp, and exports individual .csv files for each class.

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

# Load Dataset

In [ ]:
# Load data
header_row_number = 4
data = pd.read_csv('/content/OpenBCI-RAW-2024-03-08_10-40-02.txt', on_bad_lines='warn', header=header_row_number)

# Remove the space at the beginning of every header name
data = data.rename(columns=lambda x: x.strip())

# Load Key logs
header_row_number = 0
key_logs = pd.read_csv('/content/KEYLOGGER_2024-03-08_10_39_21.csv', on_bad_lines='warn', header=header_row_number)

# Clean Data

In [ ]:
columns_to_keep = ['Sample Index',
                  'EXG Channel 0',
                  'EXG Channel 1',
                  'EXG Channel 2',
                  'EXG Channel 3',
                  'EXG Channel 4',
                  'EXG Channel 5',
                  'EXG Channel 6',
                  'EXG Channel 7',
                   'Timestamp (Formatted)']

# Remove unnecessary columns
data = data[columns_to_keep]

# Drop N/A values and count the number of dropped rows
dropped_data = data.dropna()
dropped_count = len(data) - len(dropped_data)
print(f'Dropped {dropped_count} N/A rows')

Dropped 0 N/A rows


# Merge Datasets

In [ ]:
# Convert timestamps to datetime objects
data['Timestamp (Formatted)'] = pd.to_datetime(data['Timestamp (Formatted)'], format='%Y-%m-%d %H:%M:%S')
key_logs['Timestamp'] = pd.to_datetime(key_logs['Timestamp'], format='%Y-%m-%d %H:%M:%S')

# Extract date and time up to seconds
data['Timestamp (Formatted)'] = data['Timestamp (Formatted)'].dt.strftime('%Y-%m-%d %H:%M:%S')
key_logs['Timestamp'] = key_logs['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Merge dataframes based on timestamp
merged_data = data.merge(key_logs, left_on='Timestamp (Formatted)', right_on='Timestamp', how='left')

# Fill NaN values in the 'Class' column with the last known class
merged_data['Class'] = merged_data['Class'].fillna(method='ffill')


# Obtain Final Dataset

In [ ]:
# Drop the unnecessary 'Timestamp' column
final_data = merged_data.drop(columns=['Sample Index', 'Timestamp', 'Timestamp (Formatted)'])

# Drop rows with NaN values in the 'Class' column
final_data.dropna(subset=['Class'], inplace=True)

# Reset the index
final_data.reset_index(drop=True, inplace=True)

# Combine Cue, Instructions, and Nothing into one class
final_data['Class'] = final_data['Class'].apply(lambda x: 'Nothing' if x in ['Cue', 'Instructions', 'Nothing'] else x)

# Get current date and time
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Save dataset
final_data.to_csv(f'final_dataset_{timestamp}.csv', index=False)

# Export Datasets per Class (Optional)

In [ ]:
# Get unique values in the 'Class' column
class_values = final_data['Class'].unique()

# Get current date and time
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Iterate over unique class values and export to CSV
for class_value in class_values:
    class_data = final_data[final_data['Class'] == class_value]
    class_data = class_data.drop(columns=['Class'])
    class_data.to_csv(f'{class_value}_{timestamp}.csv', index=False)

print("CSV files exported successfully!")

# New Version
## April 19, 2024

In [38]:
import pandas as pd

header_row_number = 0
class_df = pd.read_csv('/content/class_timestamps_1713489357.6326618.csv', on_bad_lines='warn')

# Upload class
data = pd.read_csv('/content/full_dataset.csv', on_bad_lines='warn', sep='\t')
data = data.iloc[:, list(range(1, 9)) + [-2]]
#data.columns = [*(f'Channel {n}' for n in range(1,9)), 'Timestamp']
data.columns = [f'Channel {n}' for n in range(1, 9)] + ['Timestamp']
#data['Class'] = None


data.head(10)

,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,Timestamp
0,-5869.031652,-5963.400717,-5914.182176,-6014.988543,-140125.768200,-147710.676218,-166853.246612,-135345.601779,1.713489e+09
1,-5870.752736,-5967.446383,-5909.376551,-6015.457930,-140125.656441,-147710.475053,-166851.078493,-135345.758241,1.713489e+09
2,-5836.733381,-5936.265699,-5914.092769,-6014.452102,-140117.699220,-147712.039675,-166861.516757,-135348.105174,1.713489e+09
3,-5832.330088,-5929.671935,-5917.490234,-6012.485148,-140116.112246,-147710.877384,-166862.790807,-135346.562904,1.713489e+09
4,-5861.812039,-5956.046993,-5913.779844,-6012.731017,-140123.130694,-147709.513928,-166853.805405,-135344.327729,1.713489e+09
5,-5875.133678,-5971.290883,-5909.823586,-6015.904965,-140127.601043,-147711.123253,-166850.832623,-135346.115869,1.713489e+09
6,-5846.746963,-5945.944005,-5917.423179,-6019.078913,-140123.935357,-147716.733541,-166864.623650,-135352.642578,1.713489e+09
7,-5835.392277,-5933.516435,-5922.899356,-6018.318953,-140121.700182,-147717.180576,-166869.384571,-135352.910799,1.713489e+09
8,-5859.643919,-5953.744764,-5918.764284,-6016.128482,-140125.768200,-147713.783111,-166859.862728,-135348.440450,1.713489e+09
9,-5877.659425,-5973.168430,-5910.829414,-6016.486110,-140128.942147,-147711.994971,-166851.458472,-135346.697014,1.713489e+09


In [39]:
merged_df = pd.merge_asof(data, class_df, on='Timestamp', direction='nearest')
merged_df.head(10)

,Channel 1,Channel 2,Channel 3,Channel 4,Channel 5,Channel 6,Channel 7,Channel 8,Timestamp,Class
0,-5869.031652,-5963.400717,-5914.182176,-6014.988543,-140125.768200,-147710.676218,-166853.246612,-135345.601779,1.713489e+09,Relax
1,-5870.752736,-5967.446383,-5909.376551,-6015.457930,-140125.656441,-147710.475053,-166851.078493,-135345.758241,1.713489e+09,Relax
2,-5836.733381,-5936.265699,-5914.092769,-6014.452102,-140117.699220,-147712.039675,-166861.516757,-135348.105174,1.713489e+09,Relax
3,-5832.330088,-5929.671935,-5917.490234,-6012.485148,-140116.112246,-147710.877384,-166862.790807,-135346.562904,1.713489e+09,Relax
4,-5861.812039,-5956.046993,-5913.779844,-6012.731017,-140123.130694,-147709.513928,-166853.805405,-135344.327729,1.713489e+09,Relax
5,-5875.133678,-5971.290883,-5909.823586,-6015.904965,-140127.601043,-147711.123253,-166850.832623,-135346.115869,1.713489e+09,Move
6,-5846.746963,-5945.944005,-5917.423179,-6019.078913,-140123.935357,-147716.733541,-166864.623650,-135352.642578,1.713489e+09,Move
7,-5835.392277,-5933.516435,-5922.899356,-6018.318953,-140121.700182,-147717.180576,-166869.384571,-135352.910799,1.713489e+09,Move
8,-5859.643919,-5953.744764,-5918.764284,-6016.128482,-140125.768200,-147713.783111,-166859.862728,-135348.440450,1.713489e+09,Move
9,-5877.659425,-5973.168430,-5910.829414,-6016.486110,-140128.942147,-147711.994971,-166851.458472,-135346.697014,1.713489e+09,Move
